<a href="https://colab.research.google.com/github/CSUC/RDR-scripts/blob/main/change_CSV_delimiter/csv_delimiter_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Change semicolon delimiter to comma
### OBSERVATION:
If you have doubts about the code, contact rdr-contacte@csuc.cat
### PURPOSE OF THE SCRIPT
The main purpose of this script is to transform the semicolon delimiter of CSV files to comma.

In [ ]:
# @title First click the &#x25B6; button to execute the script. </p> Then clic the <strong>'Upload'</strong> button to upload the CSV files. </p>
import os
import csv
import shutil
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from io import StringIO

# Function to process the CSV file
def process_csv(csv_content):
    output_csv = StringIO()

    # Read the input with the correct delimiter
    csv_reader = csv.reader(StringIO(csv_content), delimiter=';')
    csv_writer = csv.writer(output_csv, delimiter=',', quoting=csv.QUOTE_MINIMAL)

    for row in csv_reader:
        csv_writer.writerow(row)

    return output_csv.getvalue()

# Function to handle file uploads
def on_file_upload(change):
    clear_output(wait=True)

    if not change.new:
        print("Please upload a file.")
        return

    uploaded_files = change.new.values()

    # Directory for processed files
    directory_name = 'processed_csv_files'
    os.makedirs(directory_name, exist_ok=True)

    for uploaded_file in uploaded_files:
        file_content = uploaded_file['content'].decode('utf-8', errors='replace')  # Decode with UTF-8
        original_filename = uploaded_file['metadata']['name']
        new_filename = os.path.splitext(original_filename)[0] + '.csv'

        # Process the CSV content
        processed_csv_content = process_csv(file_content)

        # Save processed content to file
        new_file_path = os.path.join(directory_name, new_filename)
        with open(new_file_path, 'w', encoding='utf-8') as f:
            f.write(processed_csv_content)

    # Create a zip file of the processed directory
    zip_file = shutil.make_archive(directory_name, 'zip', directory_name)

    # Trigger the download
    files.download(zip_file)

# Display upload instructions
display(HTML("<p style='font-size:14px;'><b>Please upload your semicolon-separated CSV files.</b></p>"))

# Create and display file upload widget
upload_button = widgets.FileUpload(accept='.csv', multiple=True)
upload_button.observe(on_file_upload, names='value')
display(upload_button)
